In [ ]:
from AutoEncoders import ReLUSAE
from Utils import createImageDataloader
from transformers import CLIPVisionModel
import torch
from torch.optim import Adam

In [ ]:
loader = createImageDataloader("/Users/mawy/Desktop/Square 512", model_name="openai/clip-vit-base-patch32", batch_size=1, shuffle=False)

model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").eval()
model.config.output_hidden_states = True

SAE = ReLUSAE(d=768, n=512, lam=1)
# FIX FINDBIAS FUNCTION - THIS IS COMPLETELY BROKEN

In [ ]:
epochs = 1 # TEST RUN
optimizer = Adam(SAE.parameters(), lr=1e-3)

for epoch in range(epochs):
    for images in loader:
        with torch.no_grad():
            
            outputs = model(pixel_values = images).hidden_states[-1][:,0]

        loss = SAE.loss(outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        SAE.renorm()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")